In [1]:
from glob import glob

In [30]:
import pandas as pd
from pyensembl import ensembl_grch38
from shutil import copyfile
for csv in glob("../*.csv"):
    print(csv)
    df = pd.read_csv(csv)

    if "Symbol" in df.columns:
        pass
    else:
        df_new = df.rename(columns={
            "Gene": "Symbol", 
            "Ensembl Transcript ID": "Ensembl_Transcript_ID", 
            "Ensembl Gene ID": "Ensembl_Gene_ID",
            "Tissue Frequency": "Tissue_Frequency",
            "Pancan Frequency": "Pancan_Frequency",
        })
        aliases = []
        for gene_id, gene_name in zip(df_new.Ensembl_Gene_ID, df_new.Symbol):
            gene = ensembl_grch38.gene_by_id(gene_id)
            if gene.name != gene_name:
                aliases.append(gene.name)
            else:
                aliases.append(None)
        df_new["Aliases"] = aliases
        cols = df_new.columns
        cols = [cols[0], cols[-1]] + list(cols[1:-1])
        local_path = csv.split("/")[-1]
        df_new = df_new[cols]
        df_new.to_csv(local_path, index=False)
        df_load = pd.read_csv(local_path)
        if len(df_load) == len(df_new) and (df_load.columns.values == df_new.columns.values).all():
            copyfile(local_path, csv)
        else:
            assert False, "Columns rearranged in saved CSV for %s" % csv

../cancer-driver-variants.csv
../cancer-driver-genes.csv
../cancer-testis-antigens.csv
../class1-mhc-presentation-pathway.csv
../interferon-response.csv
